In [1]:
import torch
from torchvision import transforms
import torchvision
from PIL import Image
from models import FaceNetModel
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from random import sample, choice
import json
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
root = '/home/khairulimam/datasets/lfw-mtcnn-182/'
df = pd.read_csv('datasets/lfw-mtcnn-182.csv')
classes = df['class'].unique()
class_row_counts = df['class'].value_counts()
images_above1_index = class_row_counts[class_row_counts > 2].index.tolist()
images_above1 = df[df['class'].isin(images_above1_index)]

FileNotFoundError: File b'datasets/lfw-mtcnn-182.csv' does not exist

In [85]:
state = torch.load('log/last_920.pth')
state['accuracy'], state['epoch']

(0.9207000000000001, 304)

In [88]:
model = FaceNetModel(128, 500, True)
model.load_state_dict(state['state_dict'])
model.to(device)
model = torch.nn.DataParallel(model)

In [22]:
trfrm = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])
topil = transforms.ToPILImage()
totensor = transforms.Compose(trfrm.transforms[:-1])

In [231]:
def get_same_notsame(size):
    notsame = []
    same = []
    while len(same) < size:
        class_ = images_above1.sample().iloc[0]['class']
        same_pairs = images_above1[images_above1['class'] == class_].sample(2)
        same.append((same_pairs.iloc[0], same_pairs.iloc[1]))
    while len(notsame) < size:
        pair1 = images_above1.sample()
        pair2 = images_above1[images_above1['class'] != pair1.iloc[0]['class']].sample()
        notsame.append((pair1.iloc[0], pair2.iloc[0]))
    return {'same': same, 'notsame': notsame}
    
def imshow(img,text=None,should_save=False):
    npimg = torchvision.utils.make_grid(img).numpy()
    plt.axis("off")
    if text:
        plt.text(220, 263, text, fontweight='bold', horizontalalignment='center',
            bbox={'facecolor':'white'})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def get_distance(timg1, timg2):
    model.eval()
    with torch.no_grad():
        embed1 = model(timg1.unsqueeze(0))
        embed2 = model(timg2.unsqueeze(0))
    return F.pairwise_distance(embed1, embed2).item()

def get_path(root, item):
    return os.path.join(root, item['name'], item['id'] + "."+item['ext'])

In [245]:
data = get_same_notsame(100)

In [238]:
threshold = .5

### Test same

In [4]:
for item in data['same']:
    a,b = map(lambda i: trfrm(Image.open(get_path(root, i))).unsqueeze(0).to(device), item)
    x0, x1 = map(lambda i: totensor(Image.open(get_path(root, i))).unsqueeze(0), item)
    embed1, embed2 = model(a), model(b)
    concatenated = torch.cat((x0,x1),0)
    euclidean_distance = F.pairwise_distance(embed1, embed2)
    name1, name2 = item[0]['name'], item[1]['name']
    not_same = euclidean_distance > threshold
    sim = ("Mirip", "Tidak Mirip")[not_same]
    greater = ("≤", ">")[not_same]
    text = f"{name1} & {name2}\nJarak: {euclidean_distance.item():.2f}\nThreshold: {threshold}\nJarak {greater} threshold = {sim}"
    # imshow(concatenated, text)

NameError: name 'data' is not defined

In [ ]:
results = []
for path in data['same']:
    path1 = path[0]
    path2 = path[1]
    img0 = Image.open(path1)
    img1 = Image.open(path2)
    x0 = trfrm(img0)
    x1 = trfrm(img1)
    distance = get_distance(x0, x1)
    results.append((path2.split('/')[-1], path1.split('/')[-1], distance))
    print(path1.split('/')[-1], '\t', path2.split('/')[-1], '\t', distance)
sortedresults = sorted(results, key=lambda k: k[-1], reverse=True)
print()
print('top 5 largest same')
for it in sortedresults[:5]:
    print(it)

In [29]:
results = []
for path in data['notsame']:
    path1 = path[0]
    path2 = path[1]
    img0 = Image.open(path1)
    img1 = Image.open(path2)
    x0 = trfrm(img0)
    x1 = trfrm(img1)
    concatenated = torch.cat((totensor(img0),totensor(img1)),0)
    concatenated.shape
    distance = get_distance(x0, x1)
    results.append((path2.split('/')[-1], path1.split('/')[-1], distance))
    print(path1.split('/')[-1], '\t', path2.split('/')[-1], '\t', distance)
sortedresults = sorted(results, key=lambda k: k[-1])
print()
print('top 5 lowest notsame')
for it in sortedresults[:5]:
    print(it)

Naji_Sabri_0005.jpg 	 Leuris_Pupo_0001.jpg 	 2.829508066177368
Linda_Baboolal_0001.jpg 	 Danny_Green_0001.jpg 	 3.1970763206481934
Debra_Shank_0001.jpg 	 Leuris_Pupo_0001.jpg 	 3.60748553276062
Carlos_De_Abreu_0001.jpg 	 Morgan_Hentzen_0001.jpg 	 3.2909352779388428
Mira_Sorvino_0001.jpg 	 Joan_Dangerfield_0001.jpg 	 1.783943772315979
Robert_Vowler_0001.jpg 	 Faisal_Iqbal_0001.jpg 	 2.209958791732788
Patricia_Russo_0001.jpg 	 Meghann_Shaughnessy_0002.jpg 	 2.8840348720550537
Dick_Devine_0001.jpg 	 Stella_Tennant_0001.jpg 	 3.415562152862549
Lucrecia_Orozco_0001.jpg 	 Dustin_Brown_0001.jpg 	 2.355517625808716
Andy_Bryant_0001.jpg 	 Kristen_Breitweiser_0003.jpg 	 3.494917154312134
Ramon_Ponce_de_Leon_0001.jpg 	 Lynn_Abraham_0001.jpg 	 3.5241920948028564
Hermando_Harton_0001.jpg 	 Boris_Henry_0001.jpg 	 3.4171695709228516
Thomas_Ulrich_0001.jpg 	 Pat_Cox_0002.jpg 	 2.6086974143981934
King_Abdullah_II_0001.jpg 	 Mark_Podlesny_0001.jpg 	 3.0868496894836426
David_Millar_0001.jpg 	 Ziwang_Xu_0

In [36]:
images_above1[images_above1['class'] != 4207].sample().iloc[0]

id       Jennifer_Capriati_0006
name          Jennifer_Capriati
ext                         jpg
class                      2507
Name: 6137, dtype: object